# Teste Não-Paramétrico de Alternativas Ordenadas de Jonckheere-Terpstra

*** RASCUNHO ***

https://www.real-statistics.com/one-way-analysis-of-variance-anova/jonckheere-terpstre-test/

https://www.lexjansen.com/pharmasug/1997/STAT_PK/st04.pdf

https://math.montana.edu/jobo/thainp/twosampb.pdf


## Estatística Mann-Whitney $U_{ij}$

In [147]:
import numpy as np
import pandas as pd

class UijStatistics:

    __statistics_matrix: np.ndarray

    def __init__(self, data: pd.DataFrame):
        self.__statistics_matrix = self.__calculate_u_ij_statistics(data)

    @property
    def values(self) -> np.ndarray:
        return self.__statistics_matrix
        
    def __calculate_u_ij_statistics(self, data: pd.DataFrame) -> np.ndarray:
        treatments_count = len(data.columns)
        statistics = np.ndarray(shape=(treatments_count, treatments_count))
        values_matrix = data.T.to_numpy()

        for (i, j), _ in np.ndenumerate(statistics):
            if i >= j:
                statistics[i, j] = 0
                continue
            statistics[i, j] = self.__calculate_ij_pair_score(values_matrix[i], values_matrix[j])
        
        return statistics

    def __calculate_ij_pair_score(self, a_values: np.ndarray, b_values: np.ndarray) -> np.number:
        score = 0
        for a_value in a_values:
            for b_value in b_values:
                if a_value <= b_value:
                    score += 1 if a_value < b_value else 0.5
        return score

## Teste de Jonckheere-Terpstra

In [148]:
import scipy.stats as scipy_stats

class JonckheereTerpstraTest:

    __data: pd.DataFrame
    __N: np.number
    __n_i: np.ndarray
    __u_ij_statistics: np.ndarray

    def __init__(self, data: pd.DataFrame):
        self.__data = data
        self.__n_i = data.count("rows")
        self.__N = self.__n_i.sum()
        self.__u_ij_statistics = UijStatistics(data).values

    @property
    def data(self) -> pd.DataFrame:
        return self.__data

    @property
    def e0_j(self) -> np.number:
        N_squared = self.__N**2
        n_squared_sum = np.sum(self.__n_i**2)
        return (N_squared - n_squared_sum) / 4

    # TODO: usar cálculo completo conforme p2 do https://www.lexjansen.com/pharmasug/1997/STAT_PK/st04.pdf ou
    #       doc do SAS https://v8doc.sas.com/sashtml/stat/chap28/sect25.htm
    @property
    def var0_j(self) -> np.number:
        N = self.__N
        n = self.__n_i
        return ((N**2) * (2 * N + 3) - np.sum((n**2) * (2 * n + 3))) / 72

    @property
    def u_ij_statistics(self) -> np.ndarray:
        return self.__u_ij_statistics

    @property
    def j_statistic(self) -> np.number:
        return np.sum(self.__u_ij_statistics)

    @property
    def z_statistic(self) -> np.number:
        return (self.j_statistic - self.e0_j) / np.sqrt(self.var0_j)

    @property
    def p_value(self) -> np.number:
        return 1 - scipy_stats.norm.cdf(abs(self.z_statistic))
    
    @property
    def summary(self) -> str:
        return f'E0(J) = {self.e0_j}, Var0(J) = {self.var0_j} ' + \
               '\n' + \
               f'J = {self.j_statistic}, Z = {self.z_statistic}, p-value = {self.p_value}'

## Exemplo

exemplo 1 do https://www.lexjansen.com/pharmasug/1997/STAT_PK/st04.pdf

In [149]:
from IPython.display import display, Latex

teste_data = pd.DataFrame(
    {
        "I": [19, 20, 60, 130],
        "II": [21, 61, 80, 129],
        "III": [40, 99, 100, 149],
        "IV": [49, 110, 151, 160],
    },
    dtype=int,
)

exemplo_1 = JonckheereTerpstraTest(teste_data)

print(exemplo_1.summary)

E0(J) = 48.0, Var0(J) = 114.66666666666667 
J = 71.0, Z = 2.1478761819581815, p-value = 0.01586179648596653


In [150]:
Latex(r'E_{0}(J) =' + f'{exemplo_1.e0_j}')

<IPython.core.display.Latex object>

In [151]:
Latex(r'Var_{0}(J) =' + f'{exemplo_1.var0_j}')

<IPython.core.display.Latex object>

In [152]:
Latex(r'\sqrt{Var_{0}(J)} =' + f'{np.sqrt(exemplo_1.var0_j)}')

<IPython.core.display.Latex object>

In [153]:
import utils

Latex(r'\begin{gather}' +
      utils.array_to_latex(exemplo_1.u_ij_statistics, r'U_{ij} =') +
      r'\end{gather}')

<IPython.core.display.Latex object>

In [154]:
Latex(r'J =' + f'{exemplo_1.e0_j}')

<IPython.core.display.Latex object>

In [155]:
Latex(r'Z =' + f'{exemplo_1.z_statistic}')

<IPython.core.display.Latex object>

In [156]:
Latex(r'pvalor =' + f'{exemplo_1.p_value}')

<IPython.core.display.Latex object>

exemplo 2 - https://www.real-statistics.com/one-way-analysis-of-variance-anova/jonckheere-terpstre-test/


In [157]:
teste_data = pd.DataFrame(
    {
        "D": [45, 35, 51, 31, 62, np.NaN],
        "A": [59, 53, 31, 47, 42, 59],
        "V": [49, 69, 52, 55, 63, np.NaN],
        "B": [72, 55, 65, 58, 61, 51],
    },
)

exemplo_2 = JonckheereTerpstraTest(teste_data)
print(exemplo_2.summary)

E0(J) = 90.5, Var0(J) = 291.9166666666667 
J = 133.5, Z = 2.5167438147457135, p-value = 0.005922245869211973


exemplo 3 - https://math.montana.edu/jobo/thainp/twosampb.pdf

In [158]:
teste_data = pd.DataFrame(
    {
        "N": [98.6, 99.5, 95.8, 93.3, 98.9, 91.1, 94.5, np.NaN],
        "U": [79.8, 82.0, 88.8, 79.6, 85.7, 81.7, 88.5, np.NaN],
        "S": [54.0, 67.0, 47.2, 71.1, 62.7, 44.8, 67.4, 80.2],
    },
)

exemplo_3 = JonckheereTerpstraTest(teste_data)

print(exemplo_3.summary)

E0(J) = 80.5, Var0(J) = 275.9166666666667 
J = 2.0, Z = -4.72585823225903, p-value = 1.145727396067997e-06
